In [1]:
import pandas as pd

import requests as rq

from bs4 import BeautifulSoup as bs

from datetime import datetime
from tqdm.notebook import tqdm
import time

In [2]:
# Функция parse_auto_links() возвращает ссылки на авто.
#
# -price_range(type list) задает диапозон цен по которым ищем авто.
#
# -num_of_pages(type int) задает количество страниц, среди которых ищем ссылки на авто.

def parse_auto_links(price_range, num_of_pages=99):
    
    # Время запуска функции
    begin_time = datetime.now()
    
    # Создаем список для нашых ссылок на авто.
    all_auto_links = []
    
    # Перебираем в цикле цены. Цена задается "от", 
    # т.е. "от 100_000 и выше".
    for price in price_range:
        
        # Создаем список, в который сложем все страницы с авто.
        all_auto_pages = []     
        
        print('Searching in price range > '+ str(price))
        
        # Заполняем созданный список страницами с авто.
        for page_number in range(1,num_of_pages+1):
            all_auto_pages.append('https://auto.ru/moskva/cars/bmw/used/?output_type=list&page={a}&price_from={b}&sort=price-asc'.format(a=page_number, b=price))
        
        # Перебираем в цилке список страниц с авто.
        # Делаем запрос страницу. После этого вытаскиваем с нее все ссылки на авто.
        for url in tqdm(all_auto_pages):
            
            response = rq.get(url)
            page = bs(response.text, 'lxml')
            search_links = page.find_all('a', class_='Link ListingItemTitle-module__link')
            
            for auto_link in search_links:
                all_auto_links.append(auto_link.get('href'))
        
        print('\n')
    print('Found {} links'.format(len(all_auto_links)))
    print('Done')
    
    # Выводим время выполнения функции
    print('Execution time: {}'.format(datetime.now()-begin_time))
    
    return all_auto_links

In [3]:
price_range = [1, 1_100_000]

auto_links = parse_auto_links(price_range)

Searching in price range > 1





Searching in price range > 1100000





Found 7065 links
Done
Execution time: 0:05:37.313236


In [4]:
time = datetime.now()

file=open('bmw_auto_links.txt','w')
file.write('Time update: ' + str(time) + '\n')
for link in auto_links:
    file.write(link+'\n')

file.close()

In [2]:
# auto_links = []

# file=open('bmw_auto_links.txt','r')

# for link in file:
#     auto_links.append(link[:-1])

# auto_links.pop(0)

In [8]:
# Функция parse_auto_data() собирает информацию об авто.
#
# -auto_links(type list) список ссылок на авто, полученый с помошью функции parse_auto_links

def parse_auto_data(auto_links):
    
    # Время запуска функции
    begin_time = datetime.now()
    
    # Создаем список словарей с данными об авто 
    all_auto = []
    
    # Список ссылок с ошибками
    error_links = []
    
    # Перебираем в цикле все ссылки на авто
    for url in tqdm(auto_links):
        
        auto_info = []
        
        #Создаем словарь с данными об авто
        auto = {}
        
        rp = rq.get(url)
        
#         #Проверяем код запроса. Если не 200 (успешный запрос), повторяем
#         while rp.status_code != 200:
#             rp = rq.get(url)
        
        rp.encoding = 'utf-8'
        page = bs(rp.text, 'lxml')
        
        ################################################################
        # Ищем название модели
        car_name = page.find_all('div', class_='CardBreadcrumbs__item')
        
        # Добавляем имя в словарь
        try:
            auto['name'] = car_name[5].text
        except:
            print('Empty car_name')
            error_links.append(url)
            
        ###############################################################
        # Парсим всю техническую информацию об авто
        info = page.find_all('span', class_='CardInfo__cell')

        # Добавляем информацию об авто в словарь
        for data in info:
            auto_info.append(data.text)
            
        # Преобразуем список с информацией об авто в словарь, где нечетный индекс это key,
        # а четный индекс value
        for i in range(0,len(auto_info)-5,2):
            auto[auto_info[i]] = auto_info[i+1]
            
        ################################################################
        # Ищем ссылку на каталог с характеристиками модели
        car_catalog_info = page.find('a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default')
        catalog_info_url = car_catalog_info.get('href')

        ################################################################
        # Ищем информацию о комплектации
        comp_info = []
        
        comp = page.find_all('li', class_='CardComplectation__itemContentEl')
        for info in comp:
            comp_info.append(info.text)
        
        auto['Комплектация'] = comp_info
        
        ################################################################
        # Ищем цену авто
        lisf_of_price = page.find_all('span', class_='OfferPriceCaption__price')
        
        # Добавляем цену в словарь
        for price in lisf_of_price:
            auto['Цена'] = price.text
            
        ###############################################################    
        # Заходим в "Характеристики модели в каталоге", чтобы найти доп. информацию
        rp_catalog = rq.get(catalog_info_url)
        
#         # Опять проверяем код запроса
#         while rp_catalog.status_code != 200:
#             rp_catalog = rq.get(url)
        
        rp_catalog.encoding = 'utf-8'
        catalog_page = bs(rp_catalog.text, 'lxml')
        
        extra_info = catalog_page.find_all('dd', class_='list-values__value')
        
        # Почему-то иногда команда возвращает пустой список, из-за этого прерывается
        # работа функции. Добавил исключение. Если список пустой, то сохраняем
        # адрес ссылки в которой не удалось собрать данные
        try:
            auto['numberOfDoors'] = extra_info[10].text
        except:
            print('Empty extra_info')
            error_links.append(url)
            
        all_auto.append(auto)
    
    print('Done')
    
    # Выводим время выполнения функции
    print('Execution time: {}'.format(datetime.now()-begin_time))
    
    return all_auto

In [292]:
auto_dataset_prt_1 = parse_auto_data(auto_links[0:1000])


Done
Execution time: 0:20:02.823638


In [5]:
auto_dataset_prt_2 = parse_auto_data(auto_links[1001:2000])


Done
Execution time: 0:30:28.947757


In [ ]:
auto_dataset_prt_3 = parse_auto_data(auto_links[2001:3000])

Empty extra_info


In [ ]:
auto_dataset_prt_4 = parse_auto_data(auto_links[3001:4000])

In [ ]:
auto_dataset_prt_5 = parse_auto_data(auto_links[5001:6000])

In [ ]:
auto_dataset_prt_6 = parse_auto_data(auto_links[6001:7000])

In [ ]:
auto_dataset_prt_7 = parse_auto_data(auto_links[7001:])

In [301]:
data_prt_1 = pd.DataFrame(data=auto_dataset_prt_1)
data_prt_1.to_csv('data_prt_1.csv')

In [6]:
data_prt_2 = pd.DataFrame(data=auto_dataset_prt_2)
data_prt_2.to_csv('data_prt_2.csv')

In [ ]:
data_prt_3 = pd.DataFrame(data=auto_dataset_prt_3)
data_prt_3.to_csv('data_prt_3.csv')

In [ ]:
data_prt_4 = pd.DataFrame(data=auto_dataset_prt_4)
data_prt_4.to_csv('data_prt_4.csv')

In [ ]:
data_prt_5 = pd.DataFrame(data=auto_dataset_prt_5)
data_prt_5.to_csv('data_prt_5.csv')

In [ ]:
data_prt_6 = pd.DataFrame(data=auto_dataset_prt_6)
data_prt_6.to_csv('data_prt_6.csv')

In [ ]:
data_prt_7 = pd.DataFrame(data=auto_dataset_prt_7)
data_prt_7.to_csv('data_prt_7.csv')